## Preparación de los datos para entrenar los modelos clásicos, creación de ventanas deslizantes, asignacion de etiquetas

**Voy a crear ventanas de 1 minuto, es decir 6 muestras. Quiero predecir si habrá hipoxemia en los próximos 5 minutos, considerando hipoxemia un valor de SPO2 < 90% por al menos un minuto.**

In [2]:
# Importar librerías necesarias
import vitaldb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from tqdm import tqdm  #barra de progreso
import pyarrow
import warnings
warnings.filterwarnings('ignore')

Después de la reunión con los médicos anestesiólogos, se realizaron cambios en las columnas que había elegido inicialmente para el dataset. Voy a ejecutar la preparación de los datos para el dataset ya filtrado, limpio y completo que tengo listo con las variables definitivas después de la validación mencionada

#### Plan a seguir:

1. Definir constantes: intervalo de muestreo, tamaño ventana, horizonte, duración mínima de evento (en pasos).

2. Agrupar por caseid. Para cada cirugía:

- Asegurar orden temporal de las filas y calcular sample_idx (0,1,2...) si no existe.

- Detectar inicios de runs de hipoxemia (run_start) de longitud ≥ 6 pasos.

- Construir ventanas que terminan en cada índice i (ventana de 6 muestras: i-5..i). Generamos ventanas con stride = 1 (una ventana por paso) siempre que exista el horizonte futuro completo (descartamos ventanas cerca del final sin horizonte completo).

- Etiquetar cada ventana: 1 si dentro de los próximos 30 pasos existe al menos UN run continuo de ≥6 muestras < 90; 0 en caso contrario.

- Extraer features por ventana (dos opciones):

    - A) Features agregadas (para modelos clásicos): media, std, min, max, último valor, pendiente en la ventana, % muestras < 90 (para SpO₂), etc. → una fila por ventana.

    - B) Ventanas crudas (para deep learning): guardar la matriz (6, n_dyn_features) y el vector estático asociado (age, asa, ane_type codificada, etc.) y la etiqueta.

3. Concatenar resultados de todas las cirugías en un windows_df (features agregadas + etiqueta + caseid + time_end) y, opcionalmente, guardar las ventanas crudas en archivos .npz o TFRecords.

4. Hacer split por caseid (ej. train/val/test 70/15/15) — nunca mezclar ventanas de la misma cirugía en conjuntos diferentes.

5. Normalización / scaling: fit del scaler en train y aplicar a val/test. Alternativa clínica: normalización por cirugía o baseline dinámico (ver notas).

6. Guardar dataset procesado en parquet / np.savez para entrenamiento posterior.

In [11]:
# Pipeline: generar ventanas de 1 min y labels con horizonte 5 min (SpO2 < 90% sostenida 60s)

# ----------------------- PARÁMETROS -----------------------
SAMPLING_INTERVAL = 10                      # segundos entre muestras
WINDOW_SEC = 60
HORIZON_SEC = 5 * 60
EVENT_SEC = 60

window_steps = int(WINDOW_SEC // SAMPLING_INTERVAL)   # = 6
horizon_steps = int(HORIZON_SEC // SAMPLING_INTERVAL) # = 30
event_steps = int(EVENT_SEC // SAMPLING_INTERVAL)     # = 6

SPO2_COL = 'Solar8000/PLETH_SPO2'
DYN_PREFIX = 'Solar8000/'
ID_COL = 'caseid'

# Columnas estáticas
STATIC_COLS = [
    'age','sex','height','weight','bmi','asa','emop',
    'approach','position', 'dltubesize', 
    'preop_htn', 'preop_dm', 'preop_ecg', 'preop_pft', 'preop_hb',
    'intraop_ppf', 'intraop_rbc', 'intraop_ffp',
    'intraop_mdz', 'intraop_ftn', 'intraop_rocu', 'intraop_vecu', 
    'intraop_eph', 'intraop_phe', 'intraop_epi', 'intraop_ca',
    'dur_op_seg', 'dur_anest_seg', 'dur_case_seg'
]

# Carpeta para guardar artefactos
OUT_DIR = 'windows_output'
os.makedirs(OUT_DIR, exist_ok=True)


# ----------------------- FUNCIONES UTILITARIAS -----------------------
def ensure_sample_index(group, sampling_interval=SAMPLING_INTERVAL):
    """Si no hay columna de tiempo, crear sample_idx y time_sec."""
    group = group.copy().reset_index(drop=True)
    if 'sample_idx' not in group.columns:
        group['sample_idx'] = np.arange(len(group))
    if 'time_sec' not in group.columns:
        group['time_sec'] = group['sample_idx'] * sampling_interval
    return group

def compute_run_start_bool(bools, min_run_len=event_steps):
    """Dada una máscara booleana (True = SpO2 < 90), devuelve array run_start_full de longitud N
       con 1 donde comienza una run de longitud >= min_run_len."""
    N = len(bools)
    if N < min_run_len:
        return np.zeros(N, dtype=int)
    conv = np.convolve(bools.astype(int), np.ones(min_run_len, dtype=int), mode='valid')
    starts = (conv == min_run_len).astype(int)   # length = N - min_run_len + 1
    run_start_full = np.zeros(N, dtype=int)
    run_start_full[:len(starts)] = starts
    return run_start_full

def extract_aggregated_features(window_df, dyn_cols):
    """Devuelve un dict con features agregadas para la ventana (por cada dyn_col)."""
    feats = {}
    x = window_df[dyn_cols].values  # shape (window_steps, n_dyn)
    steps = x.shape[0]
    idx = np.arange(steps)
    for j, col in enumerate(dyn_cols):
        vals = x[:, j]
        base_name = col.split('/')[-1]  # e.g. PLETH_SPO2
        feats[f'{base_name}_mean'] = float(np.nanmean(vals))
        feats[f'{base_name}_std']  = float(np.nanstd(vals))
        feats[f'{base_name}_min']  = float(np.nanmin(vals))
        feats[f'{base_name}_max']  = float(np.nanmax(vals))
        feats[f'{base_name}_last'] = float(vals[-1])
        # slope (pendiente) por fit lineal simple
        if np.all(np.isfinite(vals)):
            try:
                slope = np.polyfit(idx, vals, 1)[0]
            except np.RankWarning:
                slope = 0.0
        else:
            slope = 0.0
        feats[f'{base_name}_slope'] = float(slope)
        # para SpO2: % tiempo por debajo de 90 en la ventana
        if base_name.upper().find('SPO2') >= 0 or base_name.upper().find('PLETH') >= 0:
            feats[f'{base_name}_pct_below_90'] = float(np.mean(vals < 90.0))
    return feats

# ----------------------- PROCESAR CADA CIRUGÍA -----------------------
def process_case_windows(group, dyn_cols, static_cols=STATIC_COLS,
                         window_steps=window_steps, horizon_steps=horizon_steps,
                         event_steps=event_steps, spo2_col=SPO2_COL):
    """
    Procesa una cirugía (group) y devuelve lista de dicts:
      - features agregadas por ventana
      - label (0/1)
      - caseid, time_end (segundos)
    """
    group = ensure_sample_index(group)
    N = len(group)
    results = []
    if N < window_steps + horizon_steps:
        # cirugía demasiado corta para crear ventanas con horizonte completo -> saltar
        return results

    # arrays veloces
    spo2 = group[spo2_col].values
    is_low = (spo2 < 90.0)
    run_start = compute_run_start_bool(is_low, min_run_len=event_steps)  # length N
    # prefix sum para queries rango de run_start
    prefix = np.concatenate(([0], run_start.cumsum()))  # length N+1, prefix[k] = sum run_start[:k]

    # índices donde la ventana puede terminar: i from window_steps-1 to N-1-horizon_steps (inclusive)
    i_start = window_steps - 1
    i_end = N - 1 - horizon_steps
    window_ends = np.arange(i_start, i_end + 1)

    # calc a,b arrays: posibles inicios de run que caben dentro del horizonte
    a = window_ends + 1
    b = window_ends + horizon_steps - event_steps + 1
    # limitar b al máximo índice válido de run_start (N - event_steps)
    max_start_index = max(0, N - event_steps)
    b = np.minimum(b, max_start_index)

    dyn = group[dyn_cols]  # pandas slice

    for i_idx, i in enumerate(window_ends):
        ai = int(a[i_idx])
        bi = int(b[i_idx])
        label = 0
        if bi >= ai:
            # número de run_starts en [ai, bi] = prefix[bi+1] - prefix[ai]
            nstarts = int(prefix[bi + 1] - prefix[ai])
            if nstarts > 0:
                label = 1
        # extraer ventana de señales para features
        win_start = i - (window_steps - 1)
        win_end = i  # inclusive
        window_df = dyn.iloc[win_start:win_end + 1]
        feats = extract_aggregated_features(window_df, dyn_cols)
        # añadir features estáticos (tomados del primer renglón de la cirugía)
        static_dict = {}
        for s in static_cols:
            if s in group.columns:
                static_dict[s] = group[s].iloc[0]
            else:
                static_dict[s] = np.nan
        # meta
        meta = {
            'caseid': group[ID_COL].iloc[0],
            'window_end_idx': int(i),
            'window_end_time_sec': float(group['time_sec'].iloc[i]),
            'label_hypoxemia': int(label)
        }
        row = {}
        row.update(meta)
        row.update(static_dict)
        row.update(feats)
        results.append(row)

    return results

# ----------------------- PIPELINE PRINCIPAL -----------------------
def build_windows_dataframe(df, dyn_prefix=DYN_PREFIX, static_cols=STATIC_COLS,
                            save_parquet=True, parquet_path=os.path.join(OUT_DIR, 'windows_aggregated2.parquet')):
    dyn_cols = [c for c in df.columns if c.startswith(dyn_prefix)]
    out_rows = []
    grouped = df.groupby(ID_COL)
    for caseid, group in tqdm(grouped, total=df[ID_COL].nunique(), desc='Procesando casos'):
        rows = process_case_windows(group, dyn_cols, static_cols)
        if rows:
            out_rows.extend(rows)
    windows_df = pd.DataFrame(out_rows)
    if save_parquet:
        windows_df.to_parquet(parquet_path, index=False)
        print(f'Saved windows aggregated to {parquet_path} (n_windows={len(windows_df)})')
    return windows_df


In [12]:
# ----------------------- EJECUCIÓN -----------------------
df = pd.read_csv('dataset_fuente_completo2.csv')
windows_df2 = build_windows_dataframe(df)
print(windows_df2.shape)
windows_df2.head() 

Procesando casos: 100%|██████████| 910/910 [13:18<00:00,  1.14it/s]


Saved windows aggregated to windows_output/windows_aggregated2.parquet (n_windows=1043478)
(1043478, 101)


,caseid,window_end_idx,window_end_time_sec,label_hypoxemia,age,sex,height,weight,bmi,asa,...,VENT_TV_min,VENT_TV_max,VENT_TV_last,VENT_TV_slope,VENT_RR_mean,VENT_RR_std,VENT_RR_min,VENT_RR_max,VENT_RR_last,VENT_RR_slope
0,7,5,50.0,1,52,F,167.7,62.3,22.2,2.0,...,893.0,893.0,893.0,9.197718e-14,0.0,0.0,0.0,0.0,0.0,0.0
1,7,6,60.0,1,52,F,167.7,62.3,22.2,2.0,...,893.0,893.0,893.0,9.197718e-14,0.0,0.0,0.0,0.0,0.0,0.0
2,7,7,70.0,1,52,F,167.7,62.3,22.2,2.0,...,893.0,893.0,893.0,9.197718e-14,0.0,0.0,0.0,0.0,0.0,0.0
3,7,8,80.0,1,52,F,167.7,62.3,22.2,2.0,...,893.0,893.0,893.0,9.197718e-14,0.0,0.0,0.0,0.0,0.0,0.0
4,7,9,90.0,1,52,F,167.7,62.3,22.2,2.0,...,893.0,893.0,893.0,9.197718e-14,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
windows_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1043478 entries, 0 to 1043477
Columns: 101 entries, caseid to VENT_RR_slope
dtypes: float64(76), int64(19), object(6)
memory usage: 804.1+ MB


In [14]:
#VERIFICACIÓN RÁPIDA INICIAL DE HIPOXEMIA
print(windows_df['label_hypoxemia'].value_counts(normalize=True))

label_hypoxemia
0    0.967905
1    0.032095
Name: proportion, dtype: float64


Ya tengo mis datos preparados, con ventanas deslizantes que tienen en cada fila los datos corresponientes a una ventana de 1 minuto y una etiqueta que dice si habrá hipoxemia en los próximos 5 minutos. Quedó almacenado en el archivo `windows_aggregated2.parquet`

Tengo un dataset con un desbalanceo grande: 3.2% positivos, 96,8% negativos. Es un desbalance habitual para casos médicos.

Con estos datos procedo a continuar con la fase de entrenamiento de modelos clásicos: Regresión Logística, Random Forest, XGBoost, Gradient Boosting (ver notebook `modelos_clasicos.ipynb`)

## Nueva versión de dataset, con cambios en la cantidad de muestras por ventana

He veido trabajando con ventanas de 1 minuto (6 muestras) para predecir hipoxemia en los próximos 5 minutos. Quiero ampliar mis ventanas y ver si esto se refleja en alguna mejora del modelo. Lo voy a hacer en dos versiones: ventanas de 5 minutos y ventanas de 10 minutos

In [15]:
# Pipeline: generar VENTANAS DE 5 MINUTOS y labels con horizonte 5 min (SpO2 < 90% sostenida 60s)

# ----------------------- PARÁMETROS -----------------------
SAMPLING_INTERVAL = 10                      # segundos entre muestras
WINDOW_SEC = 300
HORIZON_SEC = 5 * 60
EVENT_SEC = 60

window_steps = int(WINDOW_SEC // SAMPLING_INTERVAL)   # = 6
horizon_steps = int(HORIZON_SEC // SAMPLING_INTERVAL) # = 30
event_steps = int(EVENT_SEC // SAMPLING_INTERVAL)     # = 6

SPO2_COL = 'Solar8000/PLETH_SPO2'
DYN_PREFIX = 'Solar8000/'
ID_COL = 'caseid'

# Columnas estáticas
STATIC_COLS = [
    'age','sex','height','weight','bmi','asa','emop',
    'approach','position', 'dltubesize', 
    'preop_htn', 'preop_dm', 'preop_ecg', 'preop_pft', 'preop_hb',
    'intraop_ppf', 'intraop_rbc', 'intraop_ffp',
    'intraop_mdz', 'intraop_ftn', 'intraop_rocu', 'intraop_vecu', 
    'intraop_eph', 'intraop_phe', 'intraop_epi', 'intraop_ca',
    'dur_op_seg', 'dur_anest_seg', 'dur_case_seg'
]

# Carpeta para guardar artefactos
OUT_DIR = 'windows_output'
os.makedirs(OUT_DIR, exist_ok=True)


# ----------------------- FUNCIONES UTILITARIAS -----------------------
def ensure_sample_index(group, sampling_interval=SAMPLING_INTERVAL):
    """Si no hay columna de tiempo, crear sample_idx y time_sec."""
    group = group.copy().reset_index(drop=True)
    if 'sample_idx' not in group.columns:
        group['sample_idx'] = np.arange(len(group))
    if 'time_sec' not in group.columns:
        group['time_sec'] = group['sample_idx'] * sampling_interval
    return group

def compute_run_start_bool(bools, min_run_len=event_steps):
    """Dada una máscara booleana (True = SpO2 < 90), devuelve array run_start_full de longitud N
       con 1 donde comienza una run de longitud >= min_run_len."""
    N = len(bools)
    if N < min_run_len:
        return np.zeros(N, dtype=int)
    conv = np.convolve(bools.astype(int), np.ones(min_run_len, dtype=int), mode='valid')
    starts = (conv == min_run_len).astype(int)   # length = N - min_run_len + 1
    run_start_full = np.zeros(N, dtype=int)
    run_start_full[:len(starts)] = starts
    return run_start_full

def extract_aggregated_features(window_df, dyn_cols):
    """Devuelve un dict con features agregadas para la ventana (por cada dyn_col)."""
    feats = {}
    x = window_df[dyn_cols].values  # shape (window_steps, n_dyn)
    steps = x.shape[0]
    idx = np.arange(steps)
    for j, col in enumerate(dyn_cols):
        vals = x[:, j]
        base_name = col.split('/')[-1]  # e.g. PLETH_SPO2
        feats[f'{base_name}_mean'] = float(np.nanmean(vals))
        feats[f'{base_name}_std']  = float(np.nanstd(vals))
        feats[f'{base_name}_min']  = float(np.nanmin(vals))
        feats[f'{base_name}_max']  = float(np.nanmax(vals))
        feats[f'{base_name}_last'] = float(vals[-1])
        # slope (pendiente) por fit lineal simple
        if np.all(np.isfinite(vals)):
            try:
                slope = np.polyfit(idx, vals, 1)[0]
            except np.RankWarning:
                slope = 0.0
        else:
            slope = 0.0
        feats[f'{base_name}_slope'] = float(slope)
        # para SpO2: % tiempo por debajo de 90 en la ventana
        if base_name.upper().find('SPO2') >= 0 or base_name.upper().find('PLETH') >= 0:
            feats[f'{base_name}_pct_below_90'] = float(np.mean(vals < 90.0))
    return feats

# ----------------------- PROCESAR CADA CIRUGÍA -----------------------
def process_case_windows(group, dyn_cols, static_cols=STATIC_COLS,
                         window_steps=window_steps, horizon_steps=horizon_steps,
                         event_steps=event_steps, spo2_col=SPO2_COL):
    """
    Procesa una cirugía (group) y devuelve lista de dicts:
      - features agregadas por ventana
      - label (0/1)
      - caseid, time_end (segundos)
    """
    group = ensure_sample_index(group)
    N = len(group)
    results = []
    if N < window_steps + horizon_steps:
        # cirugía demasiado corta para crear ventanas con horizonte completo -> saltar
        return results

    # arrays veloces
    spo2 = group[spo2_col].values
    is_low = (spo2 < 90.0)
    run_start = compute_run_start_bool(is_low, min_run_len=event_steps)  # length N
    # prefix sum para queries rango de run_start
    prefix = np.concatenate(([0], run_start.cumsum()))  # length N+1, prefix[k] = sum run_start[:k]

    # índices donde la ventana puede terminar: i from window_steps-1 to N-1-horizon_steps (inclusive)
    i_start = window_steps - 1
    i_end = N - 1 - horizon_steps
    window_ends = np.arange(i_start, i_end + 1)

    # calc a,b arrays: posibles inicios de run que caben dentro del horizonte
    a = window_ends + 1
    b = window_ends + horizon_steps - event_steps + 1
    # limitar b al máximo índice válido de run_start (N - event_steps)
    max_start_index = max(0, N - event_steps)
    b = np.minimum(b, max_start_index)

    dyn = group[dyn_cols]  # pandas slice

    for i_idx, i in enumerate(window_ends):
        ai = int(a[i_idx])
        bi = int(b[i_idx])
        label = 0
        if bi >= ai:
            # número de run_starts en [ai, bi] = prefix[bi+1] - prefix[ai]
            nstarts = int(prefix[bi + 1] - prefix[ai])
            if nstarts > 0:
                label = 1
        # extraer ventana de señales para features
        win_start = i - (window_steps - 1)
        win_end = i  # inclusive
        window_df = dyn.iloc[win_start:win_end + 1]
        feats = extract_aggregated_features(window_df, dyn_cols)
        # añadir features estáticos (tomados del primer renglón de la cirugía)
        static_dict = {}
        for s in static_cols:
            if s in group.columns:
                static_dict[s] = group[s].iloc[0]
            else:
                static_dict[s] = np.nan
        # meta
        meta = {
            'caseid': group[ID_COL].iloc[0],
            'window_end_idx': int(i),
            'window_end_time_sec': float(group['time_sec'].iloc[i]),
            'label_hypoxemia': int(label)
        }
        row = {}
        row.update(meta)
        row.update(static_dict)
        row.update(feats)
        results.append(row)

    return results

# ----------------------- PIPELINE PRINCIPAL -----------------------
def build_windows_dataframe(df, dyn_prefix=DYN_PREFIX, static_cols=STATIC_COLS,
                            save_parquet=True, parquet_path=os.path.join(OUT_DIR, 'windows_aggregated_5min.parquet')):
    dyn_cols = [c for c in df.columns if c.startswith(dyn_prefix)]
    out_rows = []
    grouped = df.groupby(ID_COL)
    for caseid, group in tqdm(grouped, total=df[ID_COL].nunique(), desc='Procesando casos'):
        rows = process_case_windows(group, dyn_cols, static_cols)
        if rows:
            out_rows.extend(rows)
    windows_df = pd.DataFrame(out_rows)
    if save_parquet:
        windows_df.to_parquet(parquet_path, index=False)
        print(f'Saved windows aggregated to {parquet_path} (n_windows={len(windows_df)})')
    return windows_df


In [16]:
# ----------------------- EJECUCIÓN -----------------------
df = pd.read_csv('dataset_fuente_completo2.csv')
windows_df3 = build_windows_dataframe(df)
print(windows_df3.shape)
windows_df3.head() 

Procesando casos: 100%|██████████| 910/910 [13:01<00:00,  1.16it/s]


Saved windows aggregated to windows_output/windows_aggregated_5min.parquet (n_windows=1021638)
(1021638, 101)


,caseid,window_end_idx,window_end_time_sec,label_hypoxemia,age,sex,height,weight,bmi,asa,...,VENT_TV_min,VENT_TV_max,VENT_TV_last,VENT_TV_slope,VENT_RR_mean,VENT_RR_std,VENT_RR_min,VENT_RR_max,VENT_RR_last,VENT_RR_slope
0,7,29,290.0,0,52,F,167.7,62.3,22.2,2.0,...,893.0,893.0,893.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,30,300.0,0,52,F,167.7,62.3,22.2,2.0,...,893.0,893.0,893.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7,31,310.0,0,52,F,167.7,62.3,22.2,2.0,...,893.0,893.0,893.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7,32,320.0,0,52,F,167.7,62.3,22.2,2.0,...,893.0,893.0,893.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7,33,330.0,0,52,F,167.7,62.3,22.2,2.0,...,893.0,893.0,893.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
windows_df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1021638 entries, 0 to 1021637
Columns: 101 entries, caseid to VENT_RR_slope
dtypes: float64(76), int64(19), object(6)
memory usage: 787.2+ MB


In [18]:
#VERIFICACIÓN RÁPIDA INICIAL DE HIPOXEMIA
print(windows_df3['label_hypoxemia'].value_counts(normalize=True))

label_hypoxemia
0    0.968042
1    0.031958
Name: proportion, dtype: float64


El dataset con ventanas que contienen la información de 5 minutos de muestras quedó almaceado en el archivo `windows_aggregated_5min.parquet`

In [19]:
# Pipeline: generar VENTANAS DE 10 MINUTOS y labels con horizonte 10 min (SpO2 < 90% sostenida 60s)

# ----------------------- PARÁMETROS -----------------------
SAMPLING_INTERVAL = 10                      # segundos entre muestras
WINDOW_SEC = 600
HORIZON_SEC = 5 * 60
EVENT_SEC = 60

window_steps = int(WINDOW_SEC // SAMPLING_INTERVAL)   # = 6
horizon_steps = int(HORIZON_SEC // SAMPLING_INTERVAL) # = 30
event_steps = int(EVENT_SEC // SAMPLING_INTERVAL)     # = 6

SPO2_COL = 'Solar8000/PLETH_SPO2'
DYN_PREFIX = 'Solar8000/'
ID_COL = 'caseid'

# Columnas estáticas
STATIC_COLS = [
    'age','sex','height','weight','bmi','asa','emop',
    'approach','position', 'dltubesize', 
    'preop_htn', 'preop_dm', 'preop_ecg', 'preop_pft', 'preop_hb',
    'intraop_ppf', 'intraop_rbc', 'intraop_ffp',
    'intraop_mdz', 'intraop_ftn', 'intraop_rocu', 'intraop_vecu', 
    'intraop_eph', 'intraop_phe', 'intraop_epi', 'intraop_ca',
    'dur_op_seg', 'dur_anest_seg', 'dur_case_seg'
]

# Carpeta para guardar artefactos
OUT_DIR = 'windows_output'
os.makedirs(OUT_DIR, exist_ok=True)


# ----------------------- FUNCIONES UTILITARIAS -----------------------
def ensure_sample_index(group, sampling_interval=SAMPLING_INTERVAL):
    """Si no hay columna de tiempo, crear sample_idx y time_sec."""
    group = group.copy().reset_index(drop=True)
    if 'sample_idx' not in group.columns:
        group['sample_idx'] = np.arange(len(group))
    if 'time_sec' not in group.columns:
        group['time_sec'] = group['sample_idx'] * sampling_interval
    return group

def compute_run_start_bool(bools, min_run_len=event_steps):
    """Dada una máscara booleana (True = SpO2 < 90), devuelve array run_start_full de longitud N
       con 1 donde comienza una run de longitud >= min_run_len."""
    N = len(bools)
    if N < min_run_len:
        return np.zeros(N, dtype=int)
    conv = np.convolve(bools.astype(int), np.ones(min_run_len, dtype=int), mode='valid')
    starts = (conv == min_run_len).astype(int)   # length = N - min_run_len + 1
    run_start_full = np.zeros(N, dtype=int)
    run_start_full[:len(starts)] = starts
    return run_start_full

def extract_aggregated_features(window_df, dyn_cols):
    """Devuelve un dict con features agregadas para la ventana (por cada dyn_col)."""
    feats = {}
    x = window_df[dyn_cols].values  # shape (window_steps, n_dyn)
    steps = x.shape[0]
    idx = np.arange(steps)
    for j, col in enumerate(dyn_cols):
        vals = x[:, j]
        base_name = col.split('/')[-1]  # e.g. PLETH_SPO2
        feats[f'{base_name}_mean'] = float(np.nanmean(vals))
        feats[f'{base_name}_std']  = float(np.nanstd(vals))
        feats[f'{base_name}_min']  = float(np.nanmin(vals))
        feats[f'{base_name}_max']  = float(np.nanmax(vals))
        feats[f'{base_name}_last'] = float(vals[-1])
        # slope (pendiente) por fit lineal simple
        if np.all(np.isfinite(vals)):
            try:
                slope = np.polyfit(idx, vals, 1)[0]
            except np.RankWarning:
                slope = 0.0
        else:
            slope = 0.0
        feats[f'{base_name}_slope'] = float(slope)
        # para SpO2: % tiempo por debajo de 90 en la ventana
        if base_name.upper().find('SPO2') >= 0 or base_name.upper().find('PLETH') >= 0:
            feats[f'{base_name}_pct_below_90'] = float(np.mean(vals < 90.0))
    return feats

# ----------------------- PROCESAR CADA CIRUGÍA -----------------------
def process_case_windows(group, dyn_cols, static_cols=STATIC_COLS,
                         window_steps=window_steps, horizon_steps=horizon_steps,
                         event_steps=event_steps, spo2_col=SPO2_COL):
    """
    Procesa una cirugía (group) y devuelve lista de dicts:
      - features agregadas por ventana
      - label (0/1)
      - caseid, time_end (segundos)
    """
    group = ensure_sample_index(group)
    N = len(group)
    results = []
    if N < window_steps + horizon_steps:
        # cirugía demasiado corta para crear ventanas con horizonte completo -> saltar
        return results

    # arrays veloces
    spo2 = group[spo2_col].values
    is_low = (spo2 < 90.0)
    run_start = compute_run_start_bool(is_low, min_run_len=event_steps)  # length N
    # prefix sum para queries rango de run_start
    prefix = np.concatenate(([0], run_start.cumsum()))  # length N+1, prefix[k] = sum run_start[:k]

    # índices donde la ventana puede terminar: i from window_steps-1 to N-1-horizon_steps (inclusive)
    i_start = window_steps - 1
    i_end = N - 1 - horizon_steps
    window_ends = np.arange(i_start, i_end + 1)

    # calc a,b arrays: posibles inicios de run que caben dentro del horizonte
    a = window_ends + 1
    b = window_ends + horizon_steps - event_steps + 1
    # limitar b al máximo índice válido de run_start (N - event_steps)
    max_start_index = max(0, N - event_steps)
    b = np.minimum(b, max_start_index)

    dyn = group[dyn_cols]  # pandas slice

    for i_idx, i in enumerate(window_ends):
        ai = int(a[i_idx])
        bi = int(b[i_idx])
        label = 0
        if bi >= ai:
            # número de run_starts en [ai, bi] = prefix[bi+1] - prefix[ai]
            nstarts = int(prefix[bi + 1] - prefix[ai])
            if nstarts > 0:
                label = 1
        # extraer ventana de señales para features
        win_start = i - (window_steps - 1)
        win_end = i  # inclusive
        window_df = dyn.iloc[win_start:win_end + 1]
        feats = extract_aggregated_features(window_df, dyn_cols)
        # añadir features estáticos (tomados del primer renglón de la cirugía)
        static_dict = {}
        for s in static_cols:
            if s in group.columns:
                static_dict[s] = group[s].iloc[0]
            else:
                static_dict[s] = np.nan
        # meta
        meta = {
            'caseid': group[ID_COL].iloc[0],
            'window_end_idx': int(i),
            'window_end_time_sec': float(group['time_sec'].iloc[i]),
            'label_hypoxemia': int(label)
        }
        row = {}
        row.update(meta)
        row.update(static_dict)
        row.update(feats)
        results.append(row)

    return results

# ----------------------- PIPELINE PRINCIPAL -----------------------
def build_windows_dataframe(df, dyn_prefix=DYN_PREFIX, static_cols=STATIC_COLS,
                            save_parquet=True, parquet_path=os.path.join(OUT_DIR, 'windows_aggregated_10min.parquet')):
    dyn_cols = [c for c in df.columns if c.startswith(dyn_prefix)]
    out_rows = []
    grouped = df.groupby(ID_COL)
    for caseid, group in tqdm(grouped, total=df[ID_COL].nunique(), desc='Procesando casos'):
        rows = process_case_windows(group, dyn_cols, static_cols)
        if rows:
            out_rows.extend(rows)
    windows_df = pd.DataFrame(out_rows)
    if save_parquet:
        windows_df.to_parquet(parquet_path, index=False)
        print(f'Saved windows aggregated to {parquet_path} (n_windows={len(windows_df)})')
    return windows_df


In [20]:
# ----------------------- EJECUCIÓN -----------------------
df = pd.read_csv('dataset_fuente_completo2.csv')
windows_df4 = build_windows_dataframe(df)
print(windows_df4.shape)
windows_df4.head() 

Procesando casos: 100%|██████████| 910/910 [12:51<00:00,  1.18it/s]


Saved windows aggregated to windows_output/windows_aggregated_10min.parquet (n_windows=994338)
(994338, 101)


,caseid,window_end_idx,window_end_time_sec,label_hypoxemia,age,sex,height,weight,bmi,asa,...,VENT_TV_min,VENT_TV_max,VENT_TV_last,VENT_TV_slope,VENT_RR_mean,VENT_RR_std,VENT_RR_min,VENT_RR_max,VENT_RR_last,VENT_RR_slope
0,7,59,590.0,0,52,F,167.7,62.3,22.2,2.0,...,893.0,893.0,893.0,1.201348e-14,0.0,0.0,0.0,0.0,0.0,0.0
1,7,60,600.0,0,52,F,167.7,62.3,22.2,2.0,...,893.0,893.0,893.0,1.201348e-14,0.0,0.0,0.0,0.0,0.0,0.0
2,7,61,610.0,0,52,F,167.7,62.3,22.2,2.0,...,893.0,893.0,893.0,1.201348e-14,0.0,0.0,0.0,0.0,0.0,0.0
3,7,62,620.0,0,52,F,167.7,62.3,22.2,2.0,...,893.0,893.0,893.0,1.201348e-14,0.0,0.0,0.0,0.0,0.0,0.0
4,7,63,630.0,0,52,F,167.7,62.3,22.2,2.0,...,893.0,893.0,893.0,1.201348e-14,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
windows_df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 994338 entries, 0 to 994337
Columns: 101 entries, caseid to VENT_RR_slope
dtypes: float64(76), int64(19), object(6)
memory usage: 766.2+ MB


In [22]:
#VERIFICACIÓN RÁPIDA INICIAL DE HIPOXEMIA
print(windows_df4['label_hypoxemia'].value_counts(normalize=True))

label_hypoxemia
0    0.967546
1    0.032454
Name: proportion, dtype: float64


El dataset con ventanas que contienen la información de 5 minutos de muestras quedó almaceado en el archivo `windows_aggregated_10min.parquet`

### Ventanas con datos crudos
Quiero probar las segunda opcion de ventanas deslizantes, con ventanas crudas en lugar de features agregados

In [25]:
# Pipeline: generar ventanas de 1 min con DATOS CRUDOS y labels con horizonte 5 min (SpO2 < 90% sostenida 60s)

# ----------------------- PARÁMETROS -----------------------
SAMPLING_INTERVAL = 10                      # segundos entre muestras
WINDOW_SEC = 60
HORIZON_SEC = 5 * 60
EVENT_SEC = 60

window_steps = int(WINDOW_SEC // SAMPLING_INTERVAL)   # = 30
horizon_steps = int(HORIZON_SEC // SAMPLING_INTERVAL) # = 30
event_steps = int(EVENT_SEC // SAMPLING_INTERVAL)     # = 6

SPO2_COL = 'Solar8000/PLETH_SPO2'
DYN_PREFIX = 'Solar8000/'
ID_COL = 'caseid'

# Columnas estáticas
STATIC_COLS = [
    'age','sex','height','weight','bmi','asa','emop',
    'approach','position', 'dltubesize', 
    'preop_htn', 'preop_dm', 'preop_ecg', 'preop_pft', 'preop_hb',
    'intraop_ppf', 'intraop_rbc', 'intraop_ffp',
    'intraop_mdz', 'intraop_ftn', 'intraop_rocu', 'intraop_vecu', 
    'intraop_eph', 'intraop_phe', 'intraop_epi', 'intraop_ca',
    'dur_op_seg', 'dur_anest_seg', 'dur_case_seg'
]

# Carpeta para guardar artefactos
OUT_DIR = 'windows_output'
os.makedirs(OUT_DIR, exist_ok=True)


# ----------------------- FUNCIONES UTILITARIAS -----------------------
def ensure_sample_index(group, sampling_interval=SAMPLING_INTERVAL):
    """Si no hay columna de tiempo, crear sample_idx y time_sec."""
    group = group.copy().reset_index(drop=True)
    if 'sample_idx' not in group.columns:
        group['sample_idx'] = np.arange(len(group))
    if 'time_sec' not in group.columns:
        group['time_sec'] = group['sample_idx'] * sampling_interval
    return group

def compute_run_start_bool(bools, min_run_len=event_steps):
    """Dada una máscara booleana (True = SpO2 < 90), devuelve array run_start_full de longitud N
       con 1 donde comienza una run de longitud >= min_run_len."""
    N = len(bools)
    if N < min_run_len:
        return np.zeros(N, dtype=int)
    conv = np.convolve(bools.astype(int), np.ones(min_run_len, dtype=int), mode='valid')
    starts = (conv == min_run_len).astype(int)   # length = N - min_run_len + 1
    run_start_full = np.zeros(N, dtype=int)
    run_start_full[:len(starts)] = starts
    return run_start_full


# ----------------------- PROCESAR CADA CIRUGÍA -----------------------
def process_case_windows(group, dyn_cols, static_cols=STATIC_COLS,
                         window_steps=window_steps, horizon_steps=horizon_steps,
                         event_steps=event_steps, spo2_col=SPO2_COL):
    """
    Procesa una cirugía (group) y devuelve lista de dicts:
      - datos crudos de todas las señales dinámicas (shape: window_steps x n_signals)
      - features estáticas
      - label (0/1)
      - caseid, time_end (segundos)
    """
    group = ensure_sample_index(group)
    N = len(group)
    results = []
    if N < window_steps + horizon_steps:
        # cirugía demasiado corta para crear ventanas con horizonte completo -> saltar
        return results

    # arrays veloces para calcular labels
    spo2 = group[spo2_col].values
    is_low = (spo2 < 90.0)
    run_start = compute_run_start_bool(is_low, min_run_len=event_steps)  # length N
    # prefix sum para queries rango de run_start
    prefix = np.concatenate(([0], run_start.cumsum()))  # length N+1, prefix[k] = sum run_start[:k]

    # índices donde la ventana puede terminar: i from window_steps-1 to N-1-horizon_steps (inclusive)
    i_start = window_steps - 1
    i_end = N - 1 - horizon_steps
    window_ends = np.arange(i_start, i_end + 1)

    # calc a,b arrays: posibles inicios de run que caben dentro del horizonte
    a = window_ends + 1
    b = window_ends + horizon_steps - event_steps + 1
    # limitar b al máximo índice válido de run_start (N - event_steps)
    max_start_index = max(0, N - event_steps)
    b = np.minimum(b, max_start_index)

    # Extraer datos estáticos una sola vez (primer renglón de la cirugía)
    static_dict = {}
    for s in static_cols:
        if s in group.columns:
            static_dict[s] = group[s].iloc[0]
        else:
            static_dict[s] = np.nan

    for i_idx, i in enumerate(window_ends):
        ai = int(a[i_idx])
        bi = int(b[i_idx])
        label = 0
        if bi >= ai:
            # número de run_starts en [ai, bi] = prefix[bi+1] - prefix[ai]
            nstarts = int(prefix[bi + 1] - prefix[ai])
            if nstarts > 0:
                label = 1
        
        # Extraer ventana de datos CRUDOS
        win_start = i - (window_steps - 1)
        win_end = i  # inclusive
        window_data = group[dyn_cols].iloc[win_start:win_end + 1].values  # shape: (window_steps, n_signals)
        
        # Crear diccionario con los datos crudos
        # Formato: cada señal se guarda como array de window_steps valores
        raw_data = {}
        for j, col in enumerate(dyn_cols):
            signal_name = col.split('/')[-1]  # e.g. PLETH_SPO2
            raw_data[f'{signal_name}_raw'] = window_data[:, j].tolist()  # convertir a lista para guardar en DataFrame
        
        # Meta información
        meta = {
            'caseid': group[ID_COL].iloc[0],
            'window_end_idx': int(i),
            'window_end_time_sec': float(group['time_sec'].iloc[i]),
            'label_hypoxemia': int(label)
        }
        
        # Combinar todo
        row = {}
        row.update(meta)
        row.update(static_dict)
        row.update(raw_data)
        results.append(row)

    return results


# ----------------------- PIPELINE PRINCIPAL -----------------------
def build_windows_dataframe_raw(df, dyn_prefix=DYN_PREFIX, static_cols=STATIC_COLS,
                            save_parquet=True, parquet_path=os.path.join(OUT_DIR, 'windows_raw_1min.parquet')):
    """
    Construye DataFrame con ventanas de datos crudos.
    Cada fila contiene:
      - metadata (caseid, window_end_idx, window_end_time_sec)
      - features estáticas (age, sex, etc.)
      - señales crudas (cada una como lista de window_steps valores)
      - label
    """
    dyn_cols = [c for c in df.columns if c.startswith(dyn_prefix)]
    out_rows = []
    grouped = df.groupby(ID_COL)
    
    for caseid, group in tqdm(grouped, total=df[ID_COL].nunique(), desc='Procesando casos'):
        rows = process_case_windows(group, dyn_cols, static_cols)
        if rows:
            out_rows.extend(rows)
    
    windows_df = pd.DataFrame(out_rows)
    
    if save_parquet:
        windows_df.to_parquet(parquet_path, index=False)
        print(f'Saved raw windows to {parquet_path} (n_windows={len(windows_df)})')
        print(f'Columnas: {list(windows_df.columns)}')
        print(f'Shape por señal: ({window_steps} timesteps)')
    
    return windows_df


# ----------------------- FUNCIÓN AUXILIAR PARA RECUPERAR DATOS -----------------------
def get_window_arrays(windows_df, dyn_prefix=DYN_PREFIX):
    """
    Convierte el DataFrame de ventanas con datos crudos de vuelta a arrays numpy.
    
    Returns:
        X: array shape (n_windows, window_steps, n_signals) - datos de señales
        y: array shape (n_windows,) - labels
        static_features: DataFrame con features estáticas
        metadata: DataFrame con metadata (caseid, time, etc.)
    """
    # Identificar columnas de señales crudas
    raw_cols = [c for c in windows_df.columns if c.endswith('_raw')]
    
    # Convertir listas a arrays
    X = np.array([windows_df[col].tolist() for col in raw_cols])  # shape: (n_signals, n_windows, window_steps)
    X = np.transpose(X, (1, 2, 0))  # shape: (n_windows, window_steps, n_signals)
    
    # Labels
    y = windows_df['label_hypoxemia'].values
    
    # Features estáticas
    static_features = windows_df[STATIC_COLS]
    
    # Metadata
    meta_cols = ['caseid', 'window_end_idx', 'window_end_time_sec']
    metadata = windows_df[meta_cols]
    
    return X, y, static_features, metadata

In [26]:
# ----------------------- EJECUCIÓN -----------------------
df = pd.read_csv('dataset_fuente_completo2.csv')
windows_df5 = build_windows_dataframe_raw(df)
print(windows_df5.shape)
windows_df5.head() 

Procesando casos: 100%|██████████| 910/910 [02:53<00:00,  5.24it/s] 


Saved raw windows to windows_output/windows_raw_1min.parquet (n_windows=1043478)
Columnas: ['caseid', 'window_end_idx', 'window_end_time_sec', 'label_hypoxemia', 'age', 'sex', 'height', 'weight', 'bmi', 'asa', 'emop', 'approach', 'position', 'dltubesize', 'preop_htn', 'preop_dm', 'preop_ecg', 'preop_pft', 'preop_hb', 'intraop_ppf', 'intraop_rbc', 'intraop_ffp', 'intraop_mdz', 'intraop_ftn', 'intraop_rocu', 'intraop_vecu', 'intraop_eph', 'intraop_phe', 'intraop_epi', 'intraop_ca', 'dur_op_seg', 'dur_anest_seg', 'dur_case_seg', 'ETCO2_raw', 'FEO2_raw', 'FIO2_raw', 'HR_raw', 'INCO2_raw', 'RR_CO2_raw', 'PLETH_SPO2_raw', 'PLETH_HR_raw', 'VENT_PIP_raw', 'VENT_TV_raw', 'VENT_RR_raw']
Shape por señal: (6 timesteps)
(1043478, 44)


,caseid,window_end_idx,window_end_time_sec,label_hypoxemia,age,sex,height,weight,bmi,asa,...,FEO2_raw,FIO2_raw,HR_raw,INCO2_raw,RR_CO2_raw,PLETH_SPO2_raw,PLETH_HR_raw,VENT_PIP_raw,VENT_TV_raw,VENT_RR_raw
0,7,5,50.0,1,52,F,167.7,62.3,22.2,2.0,...,"[21.0, 21.0, 21.0, 21.0, 21.0, 21.0]","[21.0, 21.0, 21.0, 21.0, 21.0, 21.0]","[65.0, 68.0, 67.0, 67.0, 67.0, 74.0]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[61.0, 61.0, 65.0, 65.0, 65.0, 65.0]","[99.0, 99.0, 101.0, 101.0, 101.0, 101.0]","[4.0, 4.0, 4.0, 4.0, 4.0, 4.0]","[893.0, 893.0, 893.0, 893.0, 893.0, 893.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,7,6,60.0,1,52,F,167.7,62.3,22.2,2.0,...,"[21.0, 21.0, 21.0, 21.0, 21.0, 21.0]","[21.0, 21.0, 21.0, 21.0, 21.0, 21.0]","[68.0, 67.0, 67.0, 67.0, 74.0, 69.0]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[61.0, 65.0, 65.0, 65.0, 65.0, 65.0]","[99.0, 101.0, 101.0, 101.0, 101.0, 101.0]","[4.0, 4.0, 4.0, 4.0, 4.0, 4.0]","[893.0, 893.0, 893.0, 893.0, 893.0, 893.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,7,7,70.0,1,52,F,167.7,62.3,22.2,2.0,...,"[21.0, 21.0, 21.0, 21.0, 21.0, 21.0]","[21.0, 21.0, 21.0, 21.0, 21.0, 21.0]","[67.0, 67.0, 67.0, 74.0, 69.0, 68.0]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[65.0, 65.0, 65.0, 65.0, 65.0, 65.0]","[101.0, 101.0, 101.0, 101.0, 101.0, 101.0]","[4.0, 4.0, 4.0, 4.0, 4.0, 4.0]","[893.0, 893.0, 893.0, 893.0, 893.0, 893.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,7,8,80.0,1,52,F,167.7,62.3,22.2,2.0,...,"[21.0, 21.0, 21.0, 21.0, 21.0, 21.0]","[21.0, 21.0, 21.0, 21.0, 21.0, 21.0]","[67.0, 67.0, 74.0, 69.0, 68.0, 69.0]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[65.0, 65.0, 65.0, 65.0, 65.0, 62.0]","[101.0, 101.0, 101.0, 101.0, 101.0, 101.0]","[4.0, 4.0, 4.0, 4.0, 4.0, 4.0]","[893.0, 893.0, 893.0, 893.0, 893.0, 893.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,7,9,90.0,1,52,F,167.7,62.3,22.2,2.0,...,"[21.0, 21.0, 21.0, 21.0, 21.0, 21.0]","[21.0, 21.0, 21.0, 21.0, 21.0, 21.0]","[67.0, 74.0, 69.0, 68.0, 69.0, 64.0]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[65.0, 65.0, 65.0, 65.0, 62.0, 59.0]","[101.0, 101.0, 101.0, 101.0, 101.0, 94.0]","[4.0, 4.0, 4.0, 4.0, 4.0, 4.0]","[893.0, 893.0, 893.0, 893.0, 893.0, 893.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


In [27]:
#VERIFICACIÓN RÁPIDA INICIAL DE HIPOXEMIA
print(windows_df5['label_hypoxemia'].value_counts(normalize=True))

label_hypoxemia
0    0.967905
1    0.032095
Name: proportion, dtype: float64


Tengo un dataset en el que la información de los datos crudos de cada ventana se almacena en listas. ESto quedó guardado en el archivo `windows_raw_1min.parquet`

In [28]:
# =====================================================================
# ESTRATEGIAS PARA TRABAJAR CON VENTANAS DE DATOS CRUDOS EN ML
# =====================================================================

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# =====================================================================
# OPCIÓN 1: Modelos de secuencias (LSTM, Transformers, CNN1D)
# =====================================================================
# Mejor para: Capturar patrones temporales complejos
# Formato necesario: (n_samples, timesteps, features)

def prepare_for_sequence_models(windows_df, dyn_prefix='Solar8000/'):
    """
    Convierte DataFrame con listas a arrays 3D para modelos de secuencias.
    
    Returns:
        X_seq: (n_windows, window_steps, n_signals) - para LSTM/CNN1D
        X_static: (n_windows, n_static_features) - features estáticas
        y: (n_windows,) - labels
    """
    # Identificar columnas
    raw_cols = [c for c in windows_df.columns if c.endswith('_raw')]
    static_cols = ['age','sex','height','weight','bmi','asa','emop',
                   'approach','position','dltubesize','preop_htn','preop_dm']
    
    # Convertir señales temporales a array 3D
    X_seq = np.array([windows_df[col].tolist() for col in raw_cols])
    X_seq = np.transpose(X_seq, (1, 2, 0))  # (n_windows, timesteps, n_signals)
    
    # Features estáticas (2D)
    X_static = windows_df[static_cols].fillna(0).values
    
    # Labels
    y = windows_df['label_hypoxemia'].values
    
    return X_seq, X_static, y


# =====================================================================
# OPCIÓN 2: Aplanar datos (flatten) para modelos tradicionales
# =====================================================================
# Mejor para: RandomForest, XGBoost, Logistic Regression
# Formato necesario: (n_samples, total_features)

def prepare_flattened(windows_df):
    """
    Aplana todas las series temporales en un vector 1D por ventana.
    
    Si tienes 30 timesteps y 5 señales = 150 features temporales
    + features estáticas = vector largo por ventana
    """
    raw_cols = [c for c in windows_df.columns if c.endswith('_raw')]
    static_cols = ['age','sex','height','weight','bmi','asa','emop',
                   'approach','position','dltubesize']
    
    # Aplanar señales temporales
    flattened_lists = []
    feature_names = []
    
    for col in raw_cols:
        signal_name = col.replace('_raw', '')
        # Convertir cada lista en múltiples columnas: signal_t0, signal_t1, ...
        arr = np.array(windows_df[col].tolist())  # (n_windows, timesteps)
        flattened_lists.append(arr)
        feature_names.extend([f'{signal_name}_t{i}' for i in range(arr.shape[1])])
    
    X_temporal = np.hstack(flattened_lists)  # (n_windows, timesteps * n_signals)
    
    # Agregar features estáticas
    X_static = windows_df[static_cols].fillna(0).values
    static_names = static_cols
    
    # Combinar todo
    X_flat = np.hstack([X_temporal, X_static])
    all_feature_names = feature_names + static_names
    
    y = windows_df['label_hypoxemia'].values
    
    return X_flat, y, all_feature_names


# =====================================================================
# OPCIÓN 3: Feature engineering manual (mejor de ambos mundos)
# =====================================================================
# Mejor para: Cuando quieres control sobre las features
# Creas features agregadas + algunas features temporales clave

def prepare_engineered_features(windows_df):
    """
    Combina estadísticas agregadas con algunas features temporales clave.
    Más eficiente que flatten completo, más interpretable que LSTM.
    """
    raw_cols = [c for c in windows_df.columns if c.endswith('_raw')]
    static_cols = ['age','sex','height','weight','bmi']
    
    features = []
    feature_names = []
    
    # Features estáticas
    X_static = windows_df[static_cols].fillna(0).values
    features.append(X_static)
    feature_names.extend(static_cols)
    
    # Para cada señal, extraer features agregadas + últimos valores
    for col in raw_cols:
        signal_name = col.replace('_raw', '')
        arr = np.array(windows_df[col].tolist())  # (n_windows, timesteps)
        
        # Estadísticas agregadas
        feat_dict = {
            f'{signal_name}_mean': np.nanmean(arr, axis=1),
            f'{signal_name}_std': np.nanstd(arr, axis=1),
            f'{signal_name}_min': np.nanmin(arr, axis=1),
            f'{signal_name}_max': np.nanmax(arr, axis=1),
            f'{signal_name}_range': np.nanmax(arr, axis=1) - np.nanmin(arr, axis=1),
        }
        
        # Últimos 5 valores (tendencia reciente)
        for i in range(5):
            feat_dict[f'{signal_name}_last_{i+1}'] = arr[:, -(i+1)]
        
        # Pendiente (trend)
        timesteps = arr.shape[1]
        x_vals = np.arange(timesteps)
        slopes = []
        for row in arr:
            if np.all(np.isfinite(row)):
                slope = np.polyfit(x_vals, row, 1)[0]
            else:
                slope = 0.0
            slopes.append(slope)
        feat_dict[f'{signal_name}_slope'] = np.array(slopes)
        
        # Agregar todas las features de esta señal
        for fname, fvals in feat_dict.items():
            features.append(fvals.reshape(-1, 1))
            feature_names.append(fname)
    
    X_engineered = np.hstack(features)
    y = windows_df['label_hypoxemia'].values
    
    return X_engineered, y, feature_names


# =====================================================================
# OPCIÓN 4: Guardar en formato optimizado para ML
# =====================================================================

def save_in_ml_format(windows_df, output_dir='ml_ready_data'):
    """
    Guarda los datos en formatos listos para ML:
    - Arrays numpy para modelos de secuencias
    - CSV aplanado para modelos tradicionales
    """
    import os
    os.makedirs(output_dir, exist_ok=True)
    
    # Formato para secuencias (LSTM, etc)
    X_seq, X_static, y = prepare_for_sequence_models(windows_df)
    np.save(f'{output_dir}/X_sequences.npy', X_seq)
    np.save(f'{output_dir}/X_static.npy', X_static)
    np.save(f'{output_dir}/y_labels.npy', y)
    
    # Formato aplanado (RF, XGBoost, etc)
    X_flat, y_flat, feature_names = prepare_flattened(windows_df)
    flat_df = pd.DataFrame(X_flat, columns=feature_names)
    flat_df['label'] = y_flat
    flat_df.to_csv(f'{output_dir}/flattened_data.csv', index=False)
    
    # Metadata
    metadata = windows_df[['caseid', 'window_end_idx', 'window_end_time_sec']]
    metadata.to_csv(f'{output_dir}/metadata.csv', index=False)
    
    print(f"✓ Guardado en {output_dir}/")
    print(f"  - X_sequences.npy: {X_seq.shape}")
    print(f"  - X_static.npy: {X_static.shape}")
    print(f"  - flattened_data.csv: {flat_df.shape}")
